In [ ]:
"""
four_tank_full.py

Four-Tank Process project
- Nonlinear model + linearization
- Visualization
- Decentralized PI design (based on RGA pairing)
- Approximate Glover–McFarlane-style loop-shaping (robustify by scaling & simple search)
- Analysis plots: tank levels, inputs, singular-values, sensitivity peak

Requires:
    numpy, scipy, matplotlib, control
"""

from typing import Callable, Tuple, Optional
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from control import ss
from numpy.linalg import inv, svd

# Physical constant
g = 981.0  # cm/s^2


# -----------------------------
# FourTank model + utilities
# -----------------------------
class FourTank:
    """
    Four-tank nonlinear model and utilities.

    States: h1, h2, h3, h4 (levels in cm)
    Inputs: u1, u2 (pump voltages, V)
    Outputs: y1, y2 (sensor voltages proportional to h1, h2)
    """

    def __init__(
        self,
        A: float = 15.52,
        kc: float = 0.2,
        k1: float = 3.3,
        k2: float = 3.3,
        a1: float = 0.071,
        a2: float = 0.057,
        a3: float = 0.071,
        a4: float = 0.057,
        gamma1: float = 0.625,
        gamma2: float = 0.625,
    ):
        self.A = float(A)
        self.kc = float(kc)
        self.k1 = float(k1)
        self.k2 = float(k2)
        self.a1 = float(a1)
        self.a2 = float(a2)
        self.a3 = float(a3)
        self.a4 = float(a4)
        self.gamma1 = float(gamma1)
        self.gamma2 = float(gamma2)

    def dynamics(self, t: float, h: np.ndarray, u: Tuple[float, float]) -> np.ndarray:
        """
        Nonlinear ODE right-hand side: dh/dt = f(h,u)
        h: array-like [h1,h2,h3,h4]
        u: tuple/list (u1,u2) or 2-element array
        Returns: [dh1, dh2, dh3, dh4]
        """
        h1, h2, h3, h4 = h
        u1, u2 = u

        # avoid sqrt of negative (numerical safety)
        h1p = max(h1, 0.0)
        h2p = max(h2, 0.0)
        h3p = max(h3, 0.0)
        h4p = max(h4, 0.0)

        dh1 = (-self.a1 / self.A) * np.sqrt(2 * g * h1p) + (self.a3 / self.A) * np.sqrt(
            2 * g * h3p
        ) + (self.gamma1 * self.k1 / self.A) * u1

        dh2 = (-self.a2 / self.A) * np.sqrt(2 * g * h2p) + (self.a4 / self.A) * np.sqrt(
            2 * g * h4p
        ) + (self.gamma2 * self.k2 / self.A) * u2

        dh3 = (-self.a3 / self.A) * np.sqrt(2 * g * h3p) + ((1.0 - self.gamma2) * self.k2 / self.A) * u2

        dh4 = (-self.a4 / self.A) * np.sqrt(2 * g * h4p) + ((1.0 - self.gamma1) * self.k1 / self.A) * u1

        return np.array([dh1, dh2, dh3, dh4], dtype=float)

    def simulate(
        self,
        h0: Tuple[float, float, float, float],
        u_func,  # either a constant 2-vector or a function u(t,h)->(u1,u2)
        t_end: float = 200.0,
        dt: float = 0.1,
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Simulate the nonlinear model.

        Returns:
            t: time vector
            h: 4 x N array of states
        """
        t_eval = np.arange(0.0, t_end + dt / 2, dt)

        def rhs(t, h):
            u = u_func(t, h) if callable(u_func) else u_func
            return self.dynamics(t, h, u)

        sol = solve_ivp(rhs, [0.0, t_end], np.asarray(h0, dtype=float), t_eval=t_eval, max_step=dt)
        return sol.t, sol.y

    def equilibrium_from_u(self, u: Tuple[float, float], t_end: float = 400.0, dt: float = 0.2) -> np.ndarray:
        """
        Approximate equilibrium by simulating long with constant input u and taking final state.
        """
        h0 = np.array([5.0, 5.0, 2.0, 2.0], dtype=float)
        _, h = self.simulate(h0, u, t_end=t_end, dt=dt)
        return h[:, -1]

    def linearize_ss(self, h_eq: np.ndarray):
        """
        Linearize around equilibrium h_eq and return state-space (A,B,C,D).
        Following the linearization structure in the lab manual:
            x = [dh1, dh2, dh3, dh4] around equilibrium
        """
        h1, h2, h3, h4 = h_eq
        # numerical safety
        h1 = max(h1, 1e-8)
        h2 = max(h2, 1e-8)
        h3 = max(h3, 1e-8)
        h4 = max(h4, 1e-8)

        # Time constants Ti = Ai/ai * sqrt(2*h0i / g) (manual)
        T1 = self.A / self.a1 * np.sqrt(2.0 * h1 / g)
        T2 = self.A / self.a2 * np.sqrt(2.0 * h2 / g)
        T3 = self.A / self.a3 * np.sqrt(2.0 * h3 / g)
        T4 = self.A / self.a4 * np.sqrt(2.0 * h4 / g)

        # A matrix following the manual's linearized structure
        A_mat = np.array(
            [
                [-1.0 / T1, 0.0, self.a3 / (self.A * T3), 0.0],
                [0.0, -1.0 / T2, 0.0, self.a4 / (self.A * T4)],
                [0.0, 0.0, -1.0 / T3, 0.0],
                [0.0, 0.0, 0.0, -1.0 / T4],
            ],
            dtype=float,
        )

        B = np.array(
            [
                [self.gamma1 * self.k1 / self.A, 0.0],
                [0.0, self.gamma2 * self.k2 / self.A],
                [0.0, (1.0 - self.gamma2) * self.k2 / self.A],
                [(1.0 - self.gamma1) * self.k1 / self.A, 0.0],
            ],
            dtype=float,
        )

        C = np.array([[self.kc, 0.0, 0.0, 0.0], [0.0, self.kc, 0.0, 0.0]], dtype=float)
        D = np.zeros((2, 2), dtype=float)

        return ss(A_mat, B, C, D)

    def transfer_matrix(self, h_eq: np.ndarray):
        """Return the linearized state-space system (2 outputs, 2 inputs)."""
        return self.linearize_ss(h_eq)

    def compute_rga(self, sys) -> np.ndarray:
        """
        Compute RGA = G(0) .* (G(0)^{-1})^T
        Handles small numerical cases (falls back to low-frequency eval).
        """
        try:
            G0 = np.array(sys.dcgain(), dtype=float)  # shape (2,2)
        except Exception:
            # fallback: evaluate at a very small frequency
            w = 1e-6
            Gf = np.squeeze(sys.freqresp([w])[0])  # shape (2,2)
            G0 = np.real(Gf)
        # compute RGA
        invG0T = inv(G0).T
        RGA = G0 * invG0T
        return RGA


# -----------------------------
# Controller design utilities
# -----------------------------
def pair_by_rga(RGA: np.ndarray):
    """
    Suggest pairing based on RGA:
    If diagonal elements are closer to 1 => use diagonal pairing (y1-u1, y2-u2).
    Otherwise use cross pairing.
    Returns: list of (y_index, u_index)
    """
    diag_pref = abs(RGA[0, 0] - 1.0) + abs(RGA[1, 1] - 1.0)
    off_pref = abs(RGA[0, 1] - 1.0) + abs(RGA[1, 0] - 1.0)
    if diag_pref <= off_pref:
        return [(0, 0), (1, 1)]
    else:
        return [(0, 1), (1, 0)]


def design_decentralized_PI(sys, pair=None, tau_i: float = 10.0):
    """
    Design simple decentralized PI controllers for the paired SISO loops.
    Heuristic:
        Kp = 0.8 / G0, Ti = tau_i
    Returns:
        Kdiag: 2x2 array of transfer-function objects (None for unused entries)
    """
    from control import tf

    if pair is None:
        pair = [(0, 0), (1, 1)]

    G0 = np.array(sys.dcgain(), dtype=float)

    Kdiag = np.empty((2, 2), dtype=object)
    Kdiag[:, :] = None

    for (yidx, uidx) in pair:
        gdc = G0[yidx, uidx]
        if abs(gdc) < 1e-9:
            Kp = 0.0
        else:
            Kp = 0.8 / gdc
        Ti = float(tau_i)
        # K(s) = Kp * (Ti*s + 1) / (Ti*s) => tf([Kp*Ti, Kp], [Ti, 0])
        K_tf = tf([Kp * Ti, Kp], [Ti, 0.0])
        Kdiag[yidx, uidx] = K_tf

    return Kdiag


# -----------------------------
# Analysis & Robustify utilities
# -----------------------------
def freq_grid(wmin: float = 1e-3, wmax: float = 1e2, points: int = 500) -> np.ndarray:
    return np.logspace(np.log10(wmin), np.log10(wmax), points)


def _eval_tf_at_omega(tf_sys, w: float) -> complex:
    """
    Evaluate a transfer function at frequency w (rad/s).
    Handles both SISO and MIMO cases.
    Returns scalar (SISO) or ndarray (MIMO).
    """
    mag, phase, omega = tf_sys.freqresp([w])
    mag = np.asarray(mag)
    phase = np.asarray(phase)

    if mag.ndim == 3:  
        # MIMO case: (nout, nin, nw)
        val = mag[:, :, 0] * np.exp(1j * phase[:, :, 0])
    elif mag.ndim == 1:  
        # SISO case: (nw,)
        val = mag[0] * np.exp(1j * phase[0])
    else:
        raise ValueError(f"Unexpected freqresp output shape: {mag.shape}")

    return np.squeeze(val)


def singular_values_at_freq(sys, Kdiag, w: float):
    """
    Evaluate singular values of L = G(jw) K(jw) and S = inv(I + L) at frequency w.
    Returns (svals_L, svals_S) where each is a 1D numpy array (length 2).
    """
    # Gf: shape (outputs, inputs, nw) returned by freqresp; take first freq index
    Gf_full = sys.freqresp([w])[0]  # shape (outputs, inputs, nw)
    Gf = np.squeeze(Gf_full[:, :, 0])  # shape (2,2)

    # Build K(jw) matrix (2x2)
    Kmat = np.zeros((2, 2), dtype=complex)
    for i in range(2):
        for j in range(2):
            Ktf = Kdiag[i, j]
            if Ktf is None:
                Kmat[i, j] = 0.0 + 0.0j
            else:
                val = _eval_tf_at_omega(Ktf, w)
                # If val is 2D matrix (rare for SISO tf object) handle:
                if np.ndim(val) == 0:
                    Kmat[i, j] = val
                else:
                    # If it's returned as shape (1,1), take entry
                    Kmat[i, j] = np.squeeze(val)

    I = np.eye(2, dtype=complex)
    L = Gf @ Kmat
    try:
        S = inv(I + L)
    except np.linalg.LinAlgError:
        S = np.full((2, 2), np.nan + 1j * np.nan)

    sL = svd(L, compute_uv=False)
    sS = svd(S, compute_uv=False) if not np.isnan(S).any() else np.array([np.nan, np.nan])
    return np.array(sL, dtype=float), np.array(sS, dtype=float)


def peak_sensitivity(sys, Kdiag, wgrid: Optional[np.ndarray] = None) -> float:
    """Return peak of the maximum singular value of S(jw) over wgrid."""
    if wgrid is None:
        wgrid = freq_grid()
    peaks = []
    for w in wgrid:
        _, sS = singular_values_at_freq(sys, Kdiag, w)
        peaks.append(np.nanmax(sS))
    return float(np.nanmax(peaks))


def robustify_by_scaling(sys, Kdiag_init, target_peak: float = 2.0, verbose: bool = True):
    """
    Approximate robustification by scaling the entire decentralized controller by scalar alpha in (0,1].
    Search logarithmically for alpha that reduces peak sensitivity below target_peak.
    Returns (Kdiag_scaled, achieved_peak, alpha_used). If target not reached, returns best found.
    """
    K0 = Kdiag_init
    alphas = np.logspace(0.0, -2.0, 40)  # 1.0 -> 0.01
    best_peak = np.inf
    best_alpha = None
    best_K = None

    for a in alphas:
        Kscaled = np.empty((2, 2), dtype=object)
        for i in range(2):
            for j in range(2):
                k = K0[i, j]
                if k is None:
                    Kscaled[i, j] = None
                else:
                    Kscaled[i, j] = a * k
        peak = peak_sensitivity(sys, Kscaled)
        if verbose:
            print(f"[robustify] alpha={a:.4f} -> peak S = {peak:.3f}")
        if peak < best_peak:
            best_peak = peak
            best_alpha = a
            best_K = Kscaled
        if peak <= target_peak:
            return Kscaled, peak, a

    return best_K, best_peak, best_alpha


# -----------------------------
# Plotting & comparison
# -----------------------------
def plot_tank_levels(t: np.ndarray, h: np.ndarray, title: str = "Tank Levels"):
    plt.figure(figsize=(10, 5))
    for i in range(4):
        plt.plot(t, h[i, :], label=f"Tank {i+1}")
    plt.xlabel("Time [s]")
    plt.ylabel("Level [cm]")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_inputs(t: np.ndarray, u_hist: np.ndarray, title: str = "Inputs"):
    plt.figure(figsize=(8, 4))
    if u_hist.ndim == 1 or u_hist.shape[0] == 1:
        plt.plot(t, u_hist, label="u")
    else:
        plt.plot(t, u_hist[0, :], label="u1")
        plt.plot(t, u_hist[1, :], label="u2")
    plt.xlabel("Time [s]")
    plt.ylabel("Voltage [V]")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_singular_values(sys, Kdiag, wgrid: Optional[np.ndarray] = None, title: str = "Singular values of L and S"):
    if wgrid is None:
        wgrid = freq_grid()
    sL1, sL2, sS1, sS2 = [], [], [], []
    for w in wgrid:
        sL, sS = singular_values_at_freq(sys, Kdiag, w)
        sL1.append(sL[0]); sL2.append(sL[1])
        sS1.append(sS[0]); sS2.append(sS[1])

    plt.figure(figsize=(10, 6))
    plt.loglog(wgrid, sL1, "--", label="sigma1(L)")
    plt.loglog(wgrid, sL2, "--", label="sigma2(L)")
    plt.loglog(wgrid, sS1, "-", label="sigma1(S)")
    plt.loglog(wgrid, sS2, "-", label="sigma2(S)")
    plt.xlabel("Frequency [rad/s]")
    plt.ylabel("Singular values")
    plt.title(title)
    plt.legend()
    plt.grid(True, which="both")
    plt.show()


# -----------------------------
# Example / Demo runner
# -----------------------------
def demo_run(show_plots: bool = True):
    """
    Demo pipeline that:
      - constructs the model
      - finds equilibrium for given u_eq
      - linearizes and computes RGA
      - designs decentralized PI
      - robustifies (scales) controllers
      - displays singular values and runs a naive closed-loop nonlinear sim

    The closed-loop nonlinear sim uses a very simple decentralized PI applied to measured lower tank voltages.
    """
    print("Four-Tank demo: simulate -> linearize -> design PI -> robustify -> compare")

    # Initialize
    ft = FourTank()
    u_eq = [5.0, 5.0]  # nominal voltages
    h_eq = ft.equilibrium_from_u(u_eq)
    print("Equilibrium levels (cm):", np.round(h_eq, 4))

    # Linearize
    sys = ft.linearize_ss(h_eq)
    RGA = ft.compute_rga(sys)
    print("RGA at DC:\n", np.round(RGA, 4))

    # Pairing
    pairing = pair_by_rga(RGA)
    print("Pairing suggested:", pairing)

    # Design PI (diagonal)
    Kdiag = design_decentralized_PI(sys, pair=pairing, tau_i=20.0)
    print("Designed decentralized PI (diagonal) controllers.")

    # Peak sensitivity
    peak_before = peak_sensitivity(sys, Kdiag)
    print(f"Peak sensitivity before robustify: {peak_before:.3f}")

    # Robustify by scaling
    K_rob, peak_after, alpha = robustify_by_scaling(sys, Kdiag, target_peak=2.0, verbose=True)
    print(f"Robustified by scaling -> alpha={alpha:.3f}, achieved peak S ~ {peak_after:.3f}")

    # Plots
    if show_plots:
        print("Plotting singular values for original controller:")
        plot_singular_values(sys, Kdiag, title="Original PI: singular values of L and S")
        print("Plotting singular values for robustified controller:")
        plot_singular_values(sys, K_rob, title="Robustified (scaled) controller: singular values of L and S")

    # Naive closed-loop nonlinear simulation using the robustified controller
    # Extract Kp and Ki heuristically from tf: assumes K(s) = Kp*(Ti*s + 1)/(Ti*s)
    from control import tf as _tf

    Kp = np.zeros((2, 2))
    Ki = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            Ktf = K_rob[i, j]
            if Ktf is None:
                Kp[i, j] = 0.0
                Ki[i, j] = 0.0
            else:
                # try to extract numerator and denominator arrays
                num = np.squeeze(Ktf.num)
                den = np.squeeze(Ktf.den)
                # expected form: num = [Kp*Ti, Kp], den = [Ti, 0]
                try:
                    if num.ndim == 1 and den.ndim == 1 and len(num) >= 2 and len(den) >= 1:
                        Kp_val = float(num[-1])
                        Ti_val = float(den[0])
                        Kp[i, j] = Kp_val
                        Ki[i, j] = (Kp_val * Ti_val) / Ti_val if Ti_val != 0 else 0.0
                    else:
                        Kp[i, j] = float(np.atleast_1d(num)[-1])
                        Ki[i, j] = 0.0
                except Exception:
                    Kp[i, j] = 0.0
                    Ki[i, j] = 0.0

    # Pairing mapping and closed-loop sim
    pair = pairing
    setpoint_cm = np.array([15.0, 15.0])  # target levels [cm]
    h0 = h_eq.copy()
    t_end = 200.0
    dt = 0.2
    tvec = np.arange(0.0, t_end + dt / 2, dt)
    h_hist = np.zeros((4, len(tvec)))
    u_hist = np.zeros((2, len(tvec)))
    integ = np.zeros(2)
    u_curr = np.array(u_eq, dtype=float)
    h = h0.copy()

    for k, tk in enumerate(tvec):
        h_hist[:, k] = h
        # measured outputs (sensor voltages) = kc * h_lower
        y = np.array([ft.kc * h[0], ft.kc * h[1]])
        sp_v = ft.kc * setpoint_cm
        e = sp_v - y  # error in sensor units (V)
        # decentralized PI (naive): each paired output drives its paired input
        du = np.zeros(2)
        for (yi, uj) in pair:
            Kp_val = Kp[yi, uj]
            Ki_val = Ki[yi, uj]
            integ[uj] += e[yi] * dt
            du[uj] += Kp_val * e[yi] + Ki_val * integ[uj]
        # update control (with simple saturation)
        u_curr = np.clip(np.array(u_eq, dtype=float) + du, 0.0, 15.0)
        u_hist[:, k] = u_curr
        # integrate dynamics for one step under piecewise-constant u_curr
        sol = solve_ivp(lambda _t, x: ft.dynamics(_t, x, u_curr), [0.0, dt], h, max_step=dt)
        h = sol.y[:, -1]

    if show_plots:
        plot_tank_levels(tvec, h_hist, title="Closed-loop levels with robustified decentralized PI (nonlinear sim)")
        plot_inputs(tvec, u_hist, title="Control inputs over time (naive decentralized PI)")

    print("Demo finished.")


if __name__ == "__main__":
    demo_run()

Four-Tank demo: simulate -> linearize -> design PI -> robustify -> compare
Equilibrium levels (cm): [27.5178 42.4784  3.8709  6.0059]
RGA at DC:
 [[ 1. -0.]
 [-0.  1.]]
Pairing suggested: [(0, 0), (1, 1)]
Designed decentralized PI (diagonal) controllers.


/home/dhruv/AirFleet-Supply-Optimizer/my-python-venv/lib/python3.13/site-packages/control/statesp.py:933: FutureWarning: StateSpace.freqresp(omega) will be removed in a future release of python-control; use sys.frequency_response(omega), or freqresp(sys, omega) in the MATLAB compatibility module instead
  warn("StateSpace.freqresp(omega) will be removed in a "
/home/dhruv/AirFleet-Supply-Optimizer/my-python-venv/lib/python3.13/site-packages/control/xferfcn.py:817: FutureWarning: TransferFunction.freqresp(omega) will be removed in a future release of python-control; use sys.frequency_response(omega), or freqresp(sys, omega) in the MATLAB compatibility module instead
  warn("TransferFunction.freqresp(omega) will be removed in a "


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed